In [1]:
from os import path, listdir
from PIL import Image
import numpy as np

In [4]:
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

1 Physical GPUs, 1 Logical GPUs


In [5]:
def get_data(directory: str) -> "tuple[np.ndarray, np.ndarray]":
    ima = listdir(path.join(directory, "PNEUMONIA"))
    nema = listdir(path.join(directory, "NORMAL"))
    size = len(ima) + len(nema)
    X = np.empty((size, 250, 350), dtype=np.uint8)
    y = np.empty((size), dtype=np.uint8)
    for i, file in enumerate(ima):
        X[i] = np.asarray(Image.open(path.join(directory, "PNEUMONIA", file)))
        y[i] = 1
    for i, file in enumerate(nema):
        X[len(ima) + i] = np.asarray(Image.open(path.join(directory, "NORMAL", file)))
        y[len(ima) + i] = 0
    return X , y

In [6]:
from modeli_stefan import make_model_3

In [9]:
directory_train = "../chest_xray_norm/train"
x_train, y_train = get_data(directory_train)

directory_val = "../chest_xray_norm/val"
x_val, y_val = get_data(directory_val)

model = make_model_3(x_train[0].shape)
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 250, 350)]        0         
_________________________________________________________________
rescaling_1 (Rescaling)      (None, 250, 350)          0         
_________________________________________________________________
gaussian_noise_1 (GaussianNo (None, 250, 350)          0         
_________________________________________________________________
tf.clip_by_value_1 (TFOpLamb (None, 250, 350)          0         
_________________________________________________________________
reshape_1 (Reshape)          (None, 250, 350, 1)       0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 248, 348, 8)       80        
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 245, 345, 12)      1548

In [ ]:
history = model.fit(x_train, y_train, epochs=350, batch_size=0, verbose=1, shuffle=True, validation_data=(x_val,y_val))

Epoch 1/350
163/163 [==============================] - 24s 141ms/step - loss: 0.4104 - accuracy: 0.8168 - precision_1: 0.8272 - recall_1: 0.9602 - val_loss: 0.7958 - val_accuracy: 0.6875 - val_precision_1: 0.6154 - val_recall_1: 1.0000
Epoch 2/350
163/163 [==============================] - 23s 143ms/step - loss: 0.1110 - accuracy: 0.9581 - precision_1: 0.9712 - recall_1: 0.9725 - val_loss: 0.4040 - val_accuracy: 0.7500 - val_precision_1: 0.6667 - val_recall_1: 1.0000
Epoch 3/350
163/163 [==============================] - 23s 138ms/step - loss: 0.0797 - accuracy: 0.9702 - precision_1: 0.9786 - recall_1: 0.9812 - val_loss: 1.5920 - val_accuracy: 0.5625 - val_precision_1: 0.5333 - val_recall_1: 1.0000
Epoch 4/350
163/163 [==============================] - 23s 140ms/step - loss: 0.0657 - accuracy: 0.9752 - precision_1: 0.9828 - recall_1: 0.9840 - val_loss: 0.6046 - val_accuracy: 0.7500 - val_precision_1: 0.6667 - val_recall_1: 1.0000
Epoch 5/350
163/163 [==============================] - 2

In [ ]:
history.history.keys()

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.figure(figsize=(10, 10))

plt.subplot(2, 2, 1)
plt.title('Loss')
plt.plot(history.epoch, history.history['loss'], label='training')
plt.plot(history.epoch, history.history['val_loss'], label='validation')
plt.legend(loc='best')

plt.subplot(2, 2, 2)
plt.title('Accuracy')
plt.plot(history.epoch, history.history['accuracy'], label='training')
plt.plot(history.epoch, history.history['val_accuracy'], label='validation')
plt.legend(loc='best')

plt.subplot(2, 2, 3)
plt.title('Precision')
plt.plot(history.epoch, history.history['precision'], label='training')
plt.plot(history.epoch, history.history['val_precision'], label='validation')
plt.legend(loc='best')

plt.subplot(2, 2, 4)
plt.title('Recall')
plt.plot(history.epoch, history.history['recall'], label='training')
plt.plot(history.epoch, history.history['val_recall'], label='validation')
plt.legend(loc='best')

plt.show()

In [ ]:
model.save("./modeli/stefan/model_2_epoha_250.h5")

In [ ]:
import json
  
# Serializing json 
json_history = json.dumps(history.history, indent = 4)
  
# Writing json
with open("./modeli/stefan/model_2_epoha_250.json", "w") as outfile:
    outfile.write(json_history)

In [ ]:
# loadovanje modela
from tensorflow.keras.models import load_model

model2 = load_model('./modeli/stefan/model_2_epoha_250.h5')

In [ ]:
#loadovanje istorije

with open('./modeli/stefan/model_2_epoha_250.json', 'r') as openfile:
    # Reading from json file
    history2 = json.load(openfile)


In [ ]:
len(history2['loss'])

In [ ]:
#history.history.keys()

In [ ]:
#len(history.history['loss'])

In [ ]:
directory_test = "../chest_xray_norm/test"
x_test, y_test = get_data(directory_test)

In [ ]:
y_pred = model2.predict(x_test)

In [ ]:
#y_pred_round = np.round(y_pred)

In [ ]:
y_pred = y_pred.astype(int)

In [ ]:
#y_test

In [ ]:
#y_pred

In [ ]:
y_pred = np.ravel(y_pred)

In [ ]:
tacnost = [y_pred==y_test]

In [ ]:
#tacnost

In [ ]:
tacnost = np.average(tacnost)

In [ ]:
tacnost